In [ ]:
!pip install datasets
!pip install jsonlines
!pip install bert_score
!pip install git+https://github.com/google-research/bleurt.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.4 MB/s eta 0:00:00
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-irqrtezg
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-irqrtezg
  Resolved https://github.com/google-research/bleurt.git to co

In [ ]:
#LLAMA
import numpy as np
import csv
import torch
import datasets
import random

filename_test = "sbic_test.csv"
filename_train = "sbic_train.csv"
bertscore_metric = datasets.load_metric("bertscore")

with open(filename_test, encoding='utf8', newline='') as file:
    csvreader = csv.reader(file)
    data_for_test = [row for row in csvreader]
    data_for_test = np.array(data_for_test[1:])

np.random.seed(2)
np.random.shuffle(data_for_test)
test_data_label = data_for_test[:,2]
test_data_post = data_for_test[:,1]
test_data_exp = data_for_test[:,-1]

random.seed(2)
prompt_index = random.sample(range(test_data_exp.shape[0]),500)

test_dict = {}
count = 0
index = 0
label = ""
exp = ""
flag = 0
accuracy = 0
c=0
L = []
bertscores = []
bertscore_metric = datasets.load_metric("bertscore")
bleurtscore_metric = datasets.load_metric("bleurt","BLEURT-20")
meteor_metric = datasets.load_metric("meteor")
bleurtscores = []
meteorscores = []
with open('llama-sbic.out', encoding='utf8', newline='') as file:
    for line in file:
        if("========" in line[:-1] and flag == 3):
            l = []
            test_dict[test_data_post[index]] = label+"#####"+exp
            l.extend([test_data_post[index],test_data_label[index],test_data_exp[index], label, exp])
            L.append(l)
            true_exp = test_data_exp[index]
            pred_exp = exp
            if (label == test_data_label[index]):
                accuracy += 1
                if(pred_exp != ''):
                    score = 0
                    for exp1 in true_exp.split(' [SEP] '):
                        score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[exp1], lang="en")["f1"][0]*100)
                    if(label == 'not offensive'):
                        score = 100
                    bertscores.append(score)
                    score = 0
                    for exp1 in true_exp.split(' [SEP] '):
                        score = max(score,bleurtscore_metric.compute(predictions=[pred_exp], references=[exp1])['scores'][0]*100)
                    if(label == 'not offensive'):
                        score = 100
                    bleurtscores.append(score)
                    score = 0
                    for exp1 in true_exp.split(' [SEP] '):
                        score = max(score,meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100)
                    if(label == 'not offensive'):
                        score = 100
                    meteorscores.append(score)
            else:
                bertscores.append(0)
                meteorscores.append(0)
                bleurtscores.append(0)
            if(label == 'offensive'):
                c+=1
            flag = 0
            count+=1
        if(line[:-1].isnumeric()):
            index = int(line[:-1])
            flag = 1
        elif ('Explanation: ' in line and flag == 1):
            exp = line.split('Explanation: ')[1][:-1]
            flag = 2
        elif ('Answer: offensive' in line and flag == 2):
            label = line.split('Answer: ')[1][:-1]
            flag = 3
        elif ('Answer: not offensive' in line and flag == 2):
            label = line.split('Answer: ')[1][:-1]
            flag = 3

import csv
with open('llama_sbic.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Post","True Label","Gold exp","Pred Label","Pred exp"])
    for l in L:
        writer.writerow(l)
print(accuracy/count)

file = open('bert_scores_llama_sbic.txt','w')
for score in bertscores:
    file.write(str(score)+"\n")
file.close()
bertscores = []


file = open('bleurt_scores_llama_sbic.txt','w')
for score in bleurtscores:
    file.write(str(score)+"\n")
file.close()
bleurtscores = []
file = open('meteor_scores_llama_sbic.txt','w')
for score in meteorscores:
    file.write(str(score)+"\n")
file.close()
meteorscores = []

<ipython-input-2-260bf2a16577>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bertscore_metric = datasets.load_metric("bertscore")


Computing checksums: 100%|##########| 1/1 [00:05<00:00,  5.98s/it]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.5576923076923077


In [ ]:
#LLAMA2
import numpy as np
import csv
import torch
import datasets
import random

filename_test = "sbic_test.csv"
filename_train = "sbic_train.csv"

with open(filename_test, encoding='utf8', newline='') as file:
    csvreader = csv.reader(file)
    data_for_test = [row for row in csvreader]
    data_for_test = np.array(data_for_test[1:])

np.random.seed(2)
np.random.shuffle(data_for_test)
test_data_label = data_for_test[:,2]
test_data_post = data_for_test[:,1]
test_data_exp = data_for_test[:,-1]

random.seed(2)
prompt_index = random.sample(range(test_data_exp.shape[0]),500)

test_dict = {}
count = 0
index = 0
label = ""
exp = ""
flag = 0
accuracy = 0
c=0
L = []
with open('llama2-sbic.out', encoding='utf8', newline='') as file:
    for line in file:
        if("========" in line[:-1] and flag == 3):
            l = []
            test_dict[test_data_post[index]] = label+"#####"+exp
            l.extend([test_data_post[index],test_data_label[index],test_data_exp[index], label, exp])
            L.append(l)
            true_exp = test_data_exp[index]
            pred_exp = exp
            if (label == test_data_label[index]):
                accuracy += 1
                if(pred_exp != ''):
                    score = 0
                    for exp1 in true_exp.split(' [SEP] '):
                        score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[exp1], lang="en")["f1"][0]*100)
                    if(label == 'not offensive'):
                        score = 100
                    bertscores.append(score)
                    score = 0
                    for exp1 in true_exp.split(' [SEP] '):
                        score = max(score,bleurtscore_metric.compute(predictions=[pred_exp], references=[exp1])['scores'][0]*100)
                    if(label == 'not offensive'):
                        score = 100
                    bleurtscores.append(score)
                    score = 0
                    for exp1 in true_exp.split(' [SEP] '):
                        score = max(score,meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100)
                    if(label == 'not offensive'):
                        score = 100
                    meteorscores.append(score)
            else:
                bertscores.append(0)
                meteorscores.append(0)
                bleurtscores.append(0)
            if(label == 'offensive'):
                c+=1
            flag = 0
            count+=1
        if(line[:-1].isnumeric()):
            index = int(line[:-1])
            flag = 1
        elif ('Explanation: ' in line and flag == 1):
            exp = line.split('Explanation: ')[1][:-1]
            flag = 2
        elif ('Answer: offensive' in line and flag == 2):
            label = line.split('Answer: ')[1][:-1]
            flag = 3
        elif ('Answer: not offensive' in line and flag == 2):
            label = line.split('Answer: ')[1][:-1]
            flag = 3
import csv
with open('llama2_sbic.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Post","True Label","Gold exp","Pred Label","Pred exp"])
    for l in L:
        writer.writerow(l)
L = []
print(accuracy/count)
file = open('bert_scores_llama2_sbic.txt','w')
for score in bertscores:
    file.write(str(score)+"\n")
file.close()
bertscores = []

file = open('bleurt_scores_llama2_sbic.txt','w')
for score in bleurtscores:
    file.write(str(score)+"\n")
file.close()
bleurtscores = []
file = open('meteor_scores_llama2_sbic.txt','w')
for score in meteorscores:
    file.write(str(score)+"\n")
file.close()
meteorscores = []

0.5918367346938775


In [ ]:
#VICUNA
import numpy as np
import csv
import torch
import datasets
import random

filename_test = "sbic_test.csv"
filename_train = "sbic_train.csv"

with open(filename_test, encoding='utf8', newline='') as file:
    csvreader = csv.reader(file)
    data_for_test = [row for row in csvreader]
    data_for_test = np.array(data_for_test[1:])

np.random.seed(2)
np.random.shuffle(data_for_test)
test_data_label = data_for_test[:,2]
test_data_post = data_for_test[:,1]
test_data_exp = data_for_test[:,-1]

random.seed(2)
prompt_index = random.sample(range(test_data_exp.shape[0]),500)

test_dict = {}
count = 0
index = 0
label = ""
exp = ""
flag = 0
accuracy = 0
c=0
with open('vicuna-sbic2.out', encoding='utf8', newline='') as file:
    for line in file:
        if("========" in line[:-1] and flag == 3):
            l = []
            test_dict[test_data_post[index]] = label+"#####"+exp
            l.extend([test_data_post[index],test_data_label[index],test_data_exp[index], label, exp])
            L.append(l)
            true_exp = test_data_exp[index]
            pred_exp = exp
            if (label == test_data_label[index]):
                accuracy += 1
                if(pred_exp != ''):
                    score = 0
                    for exp1 in true_exp.split(' [SEP] '):
                        score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[exp1], lang="en")["f1"][0]*100)
                    if(label == 'not offensive'):
                        score = 100
                    bertscores.append(score)
                    score = 0
                    for exp1 in true_exp.split(' [SEP] '):
                        score = max(score,bleurtscore_metric.compute(predictions=[pred_exp], references=[exp1])['scores'][0]*100)
                    if(label == 'not offensive'):
                        score = 100
                    bleurtscores.append(score)
                    score = 0
                    for exp1 in true_exp.split(' [SEP] '):
                        score = max(score,meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100)
                    if(label == 'not offensive'):
                        score = 100
                    meteorscores.append(score)
            else:
                bertscores.append(0)
                meteorscores.append(0)
                bleurtscores.append(0)
            if(label == 'offensive'):
                c+=1
            flag = 0
            count+=1
        if(line[:-1].isnumeric()):
            index = int(line[:-1])
            flag = 1
        elif ('Explanation: ' in line and flag == 1):
            exp = line.split('Explanation: ')[1][:-1]
            flag = 2
        elif ('Answer: offensive' in line and flag == 2):
            label = line.split('Answer: ')[1][:-1]
            flag = 3
        elif ('Answer: not offensive' in line and flag == 2):
            label = line.split('Answer: ')[1][:-1]
            flag = 3
import csv
with open('vicuna_sbic.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Post","True Label","Gold exp","Pred Label","Pred exp"])
    for l in L:
        writer.writerow(l)
L = []
print(accuracy/count)
file = open('bert_scores_vicuna_sbic.txt','w')
for score in bertscores:
    file.write(str(score)+"\n")
file.close()
bertscores = []

file = open('bleurt_scores_vicuna_sbic.txt','w')
for score in bleurtscores:
    file.write(str(score)+"\n")
file.close()
bleurtscores = []
file = open('meteor_scores_vicuna_sbic.txt','w')
for score in meteorscores:
    file.write(str(score)+"\n")
file.close()
meteorscores = []

0.5784313725490197


In [ ]:
#GPT3.5
import numpy as np
import csv
import torch
import datasets
import random

filename_test = "sbic_test.csv"
filename_train = "sbic_train.csv"

with open(filename_test, encoding='utf8', newline='') as file:
    csvreader = csv.reader(file)
    data_for_test = [row for row in csvreader]
    data_for_test = np.array(data_for_test[1:])

np.random.seed(2)
np.random.shuffle(data_for_test)
test_data_label = data_for_test[:,2]
test_data_post = data_for_test[:,1]
test_data_exp = data_for_test[:,-1]

random.seed(2)
prompt_index = random.sample(range(test_data_exp.shape[0]),500)

test_dict = {}
count = 0
index = 0
label = ""
exp = ""
flag = 0
accuracy = 0
c=0
with open('gpt-35-sbic3.out', encoding='utf8', newline='') as file:
    for line in file:
        if("========" in line[:-1] and flag == 3):
            l = []
            test_dict[test_data_post[index]] = label+"#####"+exp
            l.extend([test_data_post[index],test_data_label[index],test_data_exp[index], label, exp])
            L.append(l)
            true_exp = test_data_exp[index]
            pred_exp = exp
            if (label == test_data_label[index]):
                accuracy += 1
                if(pred_exp != ''):
                    score = 0
                    for exp1 in true_exp.split(' [SEP] '):
                        score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[exp1], lang="en")["f1"][0]*100)
                    if(label == 'not offensive'):
                        score = 100
                    bertscores.append(score)
                    score = 0
                    for exp1 in true_exp.split(' [SEP] '):
                        score = max(score,bleurtscore_metric.compute(predictions=[pred_exp], references=[exp1])['scores'][0]*100)
                    if(label == 'not offensive'):
                        score = 100
                    bleurtscores.append(score)
                    score = 0
                    for exp1 in true_exp.split(' [SEP] '):
                        score = max(score,meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100)
                    if(label == 'not offensive'):
                        score = 100
                    meteorscores.append(score)
            else:
                bertscores.append(0)
                meteorscores.append(0)
                bleurtscores.append(0)
            if(label == 'offensive'):
                c+=1
            flag = 0
            count+=1
        if(line[:-1].isnumeric()):
            index = int(line[:-1])
            flag = 1
        elif ('Explanation: ' in line):
            exp = line.split('Explanation: ')[1][:-1]
            flag = 3
        elif ('Answer: offensive' in line):
            label = line.split('Answer: ')[1][:-1]
            flag = 3
        elif ('Answer: not offensive' in line):
            label = line.split('Answer: ')[1][:-1]
            flag = 3
import csv
with open('gpt35_sbic.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Post","True Label","Gold exp","Pred Label","Pred exp"])
    for l in L:
        writer.writerow(l)
L = []
print(accuracy/count)
print(accuracy/count)
file = open('bert_scores_gpt35_sbic.txt','w')
for score in bertscores:
    file.write(str(score)+"\n")
file.close()
bertscores = []

file = open('bleurt_scores_gpt35_sbic.txt','w')
for score in bleurtscores:
    file.write(str(score)+"\n")
file.close()
bleurtscores = []
file = open('meteor_scores_gpt35_sbic.txt','w')
for score in meteorscores:
    file.write(str(score)+"\n")
file.close()
meteorscores = []

0.7869565217391304
0.7869565217391304


In [ ]:
#GPT4
import numpy as np
import csv
import torch
import datasets
import random

filename_test = "sbic_test.csv"
filename_train = "sbic_train.csv"

with open(filename_test, encoding='utf8', newline='') as file:
    csvreader = csv.reader(file)
    data_for_test = [row for row in csvreader]
    data_for_test = np.array(data_for_test[1:])

np.random.seed(2)
np.random.shuffle(data_for_test)
test_data_label = data_for_test[:,2]
test_data_post = data_for_test[:,1]
test_data_exp = data_for_test[:,-1]

random.seed(2)
prompt_index = random.sample(range(test_data_exp.shape[0]),500)

test_dict = {}
count = 0
index = 0
label = ""
exp = ""
flag = 0
accuracy = 0
c=0
with open('gpt4-sbic4.out', encoding='utf8', newline='') as file:
    for line in file:
        if("========" in line[:-1] and flag == 3):
            l = []
            test_dict[test_data_post[index]] = label+"#####"+exp
            l.extend([test_data_post[index],test_data_label[index],test_data_exp[index], label, exp])
            L.append(l)
            true_exp = test_data_exp[index]
            pred_exp = exp
            if (label == test_data_label[index]):
                accuracy += 1
                if(pred_exp != ''):
                    score = 0
                    for exp1 in true_exp.split(' [SEP] '):
                        score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[exp1], lang="en")["f1"][0]*100)
                    if(label == 'not offensive'):
                        score = 100
                    bertscores.append(score)
                    score = 0
                    for exp1 in true_exp.split(' [SEP] '):
                        score = max(score,bleurtscore_metric.compute(predictions=[pred_exp], references=[exp1])['scores'][0]*100)
                    if(label == 'not offensive'):
                        score = 100
                    bleurtscores.append(score)
                    score = 0
                    for exp1 in true_exp.split(' [SEP] '):
                        score = max(score,meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100)
                    if(label == 'not offensive'):
                        score = 100
                    meteorscores.append(score)
            else:
                bertscores.append(0)
                meteorscores.append(0)
                bleurtscores.append(0)
            if(label == 'offensive'):
                c+=1
            flag = 0
            count+=1
        if(line[:-1].isnumeric()):
            index = int(line[:-1])
            flag = 1
        elif ('Explanation: ' in line):
            exp = line.split('Explanation: ')[1][:-1]
            flag = 3
        elif ('Answer: offensive' in line):
            label = line.split('Answer: ')[1][:-1]
            flag = 3
        elif ('Answer: not offensive' in line):
            label = line.split('Answer: ')[1][:-1]
            flag = 3
import csv
with open('gpt4_sbic.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Post","True Label","Gold exp","Pred Label","Pred exp"])
    for l in L:
        writer.writerow(l)
L = []
print(accuracy/count)
print(accuracy/count)
file = open('bert_scores_gpt4_sbic.txt','w')
for score in bertscores:
    file.write(str(score)+"\n")
file.close()
bertscores = []
file = open('bleurt_scores_gpt4_sbic.txt','w')
for score in bleurtscores:
    file.write(str(score)+"\n")
file.close()
bleurtscores = []
file = open('meteor_scores_gpt4_sbic.txt','w')
for score in meteorscores:
    file.write(str(score)+"\n")
file.close()
meteorscores = []

0.858
0.858


In [ ]:
scores = []
file = open('bert_scores_llama_sbic.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bert_scores_llama2_sbic.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bert_scores_vicuna_sbic.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bert_scores_gpt35_sbic.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bert_scores_gpt4_sbic.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))

51.476594507694244
55.96229015564432
54.10707850082248
74.53748115285515
81.71392897367477
33.511440081357534


In [ ]:

scores = []
file = open('bleurt_scores_llama_sbic.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bleurt_scores_llama2_sbic.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bleurt_scores_vicuna_sbic.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bleurt_scores_gpt35_sbic.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bleurt_scores_gpt4_sbic.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))

25.256179353365532
36.75341604315505
36.637674661633234
57.21965417851527
62.72095550298691


In [ ]:


scores = []
file = open('meteor_scores_llama_sbic.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('meteor_scores_llama2_sbic.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('meteor_scores_vicuna_sbic.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('meteor_scores_gpt35_sbic.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('meteor_scores_gpt4_sbic.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))

14.218843947997005
23.695966667746948
25.831860703112866
42.746439241180404
45.08957475622339
